In [4]:
import sqlite3

# conexión a la bbdd
conn1=sqlite3.connect('rrss ampliado.db')
c1=conn1.cursor()


# PASO 1
# Borrar los datos que no están en INCLUIDO (se han añadido por el nombre de usuario y no por el contenido)
idpost_principal = c1.execute('SELECT id_post FROM PRINCIPAL').fetchall()
idpost_principal = [item[0] for item in idpost_principal]
idpost_incluido = c1.execute('SELECT id_principal FROM INCLUIDO').fetchall()
idpost_incluido = [item[0] for item in idpost_incluido]
i=0
lista_borrar=[]
for elemento in idpost_principal:
    if elemento not in idpost_incluido:
        lista_borrar.append(elemento)
        i=i+1     
print('Comentarios que no están en incluido:',i)


# PASO 2 
# Borrar los comentarios anteriores a 03/2022
idprincipal_antiguos = c1.execute("""SELECT id_post FROM PRINCIPAL WHERE fecha<'2022-03-01%';""").fetchall()
idprincipal_antiguos = [item[0] for item in idprincipal_antiguos]
i=0
for elemento in idprincipal_antiguos:
    if elemento not in lista_borrar:
        lista_borrar.append(elemento)
        i=i+1
print('Comentarios anteriores a marzo de 2022:',i)


# PASO 3 
# Borrar los comentarios largos
idprincipal_largos = c1.execute("""SELECT id_post FROM PRINCIPAL WHERE length(contenido)>500;""").fetchall()
idprincipal_largos = [item[0] for item in idprincipal_largos]
i=0
for elemento in idprincipal_largos:
    if elemento not in lista_borrar:
        lista_borrar.append(elemento)
        i=i+1
print('Comentarios largos:',i)


# PASO 4
# Borrar los comentarios repetidos más de 3 veces
contenido_repetido = c1.execute("""SELECT contenido FROM PRINCIPAL group by contenido HAVING COUNT(contenido)>3;""").fetchall()
contenido_repetido = [item[0] for item in contenido_repetido]
i=0
for elemento in contenido_repetido:
    id_cont_repetido= c1.execute("""SELECT id_post FROM PRINCIPAL WHERE contenido=(?);""",(elemento,)).fetchall()
    id_cont_repetido = [item[0] for item in id_cont_repetido]
    for id in id_cont_repetido:
        if id not in lista_borrar:
            lista_borrar.append(id)
            i=i+1
print('Comentarios repetidos más de 3 veces:',i)


# PASO 5 
# Borrar comentarios con palabras que identificamos como spam
spam=['instant cash','giveaway','give away','giving away','app get','[H]','winner','make a millionaire','rt + done',
      'earn money','prizes','free nft for','BinanceVisit','rewards','sign up', 'referral','metavip','saiyan',
      'tag 3','rt + ','[store]','Mana Ooru','get a free','tradingview','free mint']
i=0
for word in spam:
    a='%'+word+'%'
    idpost_spam = c1.execute("""SELECT id_post FROM PRINCIPAL WHERE contenido like (?);""",(a,)).fetchall()
    idpost_spam = [item[0] for item in idpost_spam]
    for id in idpost_spam:
        if id not in lista_borrar:
            lista_borrar.append(id)
            i=i+1
print('Comentarios potencialmente spam:',i)


# PASO 6
# Borrar los comentarios que no estén en inglés
import emoji
import re
from langdetect import detect
i=0
for id in idpost_principal:
    if id not in lista_borrar:
        a=c1.execute("""SELECT contenido FROM PRINCIPAL WHERE id_post=(?);""",(id,)).fetchall()
        conn1.commit()   
        a = [item[0] for item in a]
        texto=emoji.get_emoji_regexp().sub("",a[0])
        texto=re.sub(r'http\S+', '',texto)
        try:
            idioma=detect(texto)
            if idioma!='en':
                if id not in lista_borrar:
                    lista_borrar.append(id)
                    i=i+1
        except:
            if id not in lista_borrar:
                lista_borrar.append(id)
                i=i+1
print('Comentarios que no están en inglés:',i)


# PASO 7
# Borrar comentarios con 0 interacciones
sin_int = c1.execute("""SELECT id_post FROM PRINCIPAL WHERE interacciones=0;""").fetchall()
sin_int = [item[0] for item in sin_int]
i=0
for elemento in sin_int:
    if elemento not in lista_borrar:
        lista_borrar.append(elemento)
        i=i+1
print('Comentarios sin interacciones:',i)


for elemento in lista_borrar:
    c1.execute("""DELETE FROM INTERACCIONES WHERE id_int=(SELECT id_int FROM CONTIENE WHERE id_principal=(?));""",(elemento,))
    conn1.commit()
    c1.execute("""DELETE FROM PRINCIPAL WHERE id_post=(?);""",(elemento,))
    conn1.commit()
    c1.execute("""DELETE FROM CONTIENE WHERE id_principal=(?);""",(elemento,))
    conn1.commit()
    c1.execute("""DELETE FROM INCLUIDO WHERE id_principal=(?);""",(elemento,))
    conn1.commit()

c1.close()
conn1.close()

print(len(lista_borrar))

Comentarios que no están en incluido: 55
55


In [3]:

import sqlite3

# conexión a la bbdd
conn1=sqlite3.connect('rrss ampliado.db')
c1=conn1.cursor()


lista_topic=['bitcoin','btc','mana','decentraland','cryptocurrency','crypto','metaverse','nft','nfts']

idpost_principal = c1.execute('SELECT id_post FROM PRINCIPAL').fetchall()
idpost_principal = [item[0] for item in idpost_principal]

for id in idpost_principal:
    contenido=c1.execute("""SELECT contenido FROM PRINCIPAL WHERE id_post=(?);""",(id,)).fetchall()
    conn1.commit()   
    contenido = [item[0] for item in contenido]
    contenido=contenido[0]
      
    for x in lista_topic:
        if x.lower() in contenido.lower():
            try:
                c1.execute("""INSERT INTO INCLUIDO VALUES((SELECT id_topic FROM TOPIC WHERE criterio=(?)), (?));""",(x,id,))
                conn1.commit()
            except sqlite3.Error as error:
                print("No se ha podido meter los datos en la tabla Incluido:", error) 
            
            
c1.close()
conn1.close()

In [44]:
import sqlite3

# conexión a la bbdd
conn1=sqlite3.connect('datos rrss limpio ampliado - copia.db')
c1=conn1.cursor()


lista_topic=['bitcoin','btc','mana','decentraland','cryptocurrency','crypto','metaverse','nft','nfts']

idpost_principal = c1.execute('SELECT id_post FROM PRINCIPAL').fetchall()
idpost_principal = [item[0] for item in idpost_principal]

borrar=[]

spam=['instant cash','giveaway','give away','giving away','app get','[H]','winner','make a mill','rt + done','join discor','okx','pre sale','mắt','wicc',
      'earn money','prizes','free nft for','BinanceVisit','rewards','sign up', 'referral','metavip','saiyan','hro n','GÜNCEL','trillioh','recipe','to all of you',
      'tag 3','rt + ','[store]','seiken','The_Art_Thieves','Mana Ooru','get a free','| crypto news today','free mint','yas','to win t','cryptocurrencyclic','ada fr','choose your']
i=0
for word in spam:
    a='%'+word+'%'
    idpost_spam = c1.execute("""SELECT id_post FROM PRINCIPAL WHERE contenido like (?);""",(a,)).fetchall()
    idpost_spam = [item[0] for item in idpost_spam]
    for id in idpost_spam:
        if id not in borrar:
            borrar.append(id)
            i=i+1
print('Comentarios potencialmente spam:',i)



# for id in idpost_principal:
#     a=c1.execute("""SELECT contenido FROM PRINCIPAL WHERE id_post=(?);""",(id,)).fetchall()
#     conn1.commit() 
#     a=[item[0] for item in a]
#     for x in lista_topic:
#         if x.lower() in a[0].lower():
#             try:
#                 c1.execute("""INSERT INTO INCLUIDO VALUES((SELECT id_topic FROM TOPIC WHERE criterio=(?)), (?));""",(x,id,))
#                 conn1.commit()
#             except sqlite3.Error as error:
#                 print("No se ha podido meter los datos en la tabla INCLUIDO:", error,id) 



# idpost_incluido = c1.execute('SELECT id_principal FROM INCLUIDO').fetchall()
# idpost_incluido = [item[0] for item in idpost_incluido]
# i=0
# for elemento in idpost_principal:
#     if elemento not in idpost_incluido:
#         borrar.append(elemento)
#         i=i+1     
# print('Comentarios que no están en incluido:',i)



for elemento in borrar:
    c1.execute("""DELETE FROM INTERACCIONES WHERE id_int=(SELECT id_int FROM CONTIENE WHERE id_principal=(?));""",(elemento,))
    conn1.commit()
    c1.execute("""DELETE FROM PRINCIPAL WHERE id_post=(?);""",(elemento,))
    conn1.commit()
    c1.execute("""DELETE FROM CONTIENE WHERE id_principal=(?);""",(elemento,))
    conn1.commit()
    c1.execute("""DELETE FROM INCLUIDO WHERE id_principal=(?);""",(elemento,))
    conn1.commit()

c1.close()
conn1.close()

print(len(borrar))

Comentarios potencialmente spam: 75
75


In [27]:
c1.close()
conn1.close()

In [ ]:
import pandas as pd
import sqlite3
import re, emoji


# Conexión a la base de datos SQLite
conn = sqlite3.connect("datos rrss.db")
c=conn.cursor()

# Se usa read_sql_query de pandas para extraer el resultado de la consulta a un DataFrame
df_3 = pd.read_sql_query("SELECT * FROM PALABRAS;", conn)
conn.commit()


for i in range(len(df_3)):
    post=df_3.loc[i,'palabras']
    post=re.sub(r'http\S+', '',post)
    post=emoji.get_emoji_regexp().sub("",post)
    post=re.sub(r'[^\w]',' ',post)




df_3.to_sql("PALABRAS", con=conn, if_exists='replace')


c.close()
conn.close()



In [10]:
import re, emoji
a='Proof of *Solace (x-post from /r/Bitcoin) 💥 https://t.co/UF59Ap53Tf'
print(a)
post1=re.sub(r'http\S+', '',a)
print(post1)
# post1=emoji.get_emoji_regexp().sub("",post1)
post1=emoji.replace_emoji(post1,replace="")
print(post1)
# post1=re.sub(r'x-post from +', '',post1)
post1=post1.split('(x-post from',1)[0]
print(post1)
post1=re.sub(r'[^\w]',' ',post1)
print(post1)

Proof of *Solace (x-post from /r/Bitcoin) 💥 https://t.co/UF59Ap53Tf
Proof of *Solace (x-post from /r/Bitcoin) 💥 
Proof of *Solace (x-post from /r/Bitcoin)  
Proof of *Solace 
Proof of  Solace 
